In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from tqdm import tqdm
import seaborn as sns
from scipy.interpolate import interp1d
import os
import sys
from matplotlib.colors import LinearSegmentedColormap

cwd = os.getcwd()
model_path = os.path.abspath(os.path.join(cwd, os.pardir, 'models'))
sys.path.append(model_path)
from cumul_diamond_ODE_model import *
from cumul_treated_diamond_ODE_model import *

Figure 3A,B

In [ ]:
untreated_model = cumulDiamondODEModel()
treated_model = cumulTreatedDiamondODEModel()

t_interventions = np.linspace(0, 30, 50)
pint_values = np.linspace(0, 1, 50)
t_final = 31
n_tot = 500
n_simulations = 100

effp = 0.7
effc = 0.7
nu = 2/4.3

inferred_parameters = pd.read_csv('../data/inferred_parameters.csv')
data = pd.read_csv('../data/data_cases_symp.csv')
data['onset_date'] = pd.to_datetime(data['onset_date'], format='%d-%b')
times = np.linspace(0, t_final, n_tot)

In [ ]:
# Sample parameter sets in advance
np.random.seed(0)
sampled_indices = np.random.randint(0, inferred_parameters.shape[0], size=n_simulations)
sampled_parameters = [inferred_parameters.iloc[idx].to_list() for idx in sampled_indices]

# Estimate median untreated cumulative infections at t_final
def simulate_untreated_once(params):
    untreated_values = untreated_model.simulate(params[:8], times)
    return untreated_values[-1, 22] + untreated_values[-1, 23]

untreated_cum_infs = Parallel(n_jobs=-1)(
    delayed(simulate_untreated_once)(params) for params in tqdm(sampled_parameters, desc='Simulating untreated')
)
median_untreated_infections = np.median(untreated_cum_infs)
print(f'Median untreated infections at day {t_final}: {median_untreated_infections:.1f}')

# Simulate treated model across grid using same parameters
def simulate_combination(t_intervention, pint):
    cum_infs = []

    for params in sampled_parameters:
        untreated_values = untreated_model.simulate(params[:8], times)

        treated_time = np.linspace(t_intervention, t_final, int((t_final - t_intervention) * n_tot / t_final))

        if t_intervention == 0:
            y0 = None
        else:
            # Interpolate untreated state at t_intervention
            interpolators = [interp1d(times, untreated_values[:, k], kind='linear') for k in range(untreated_values.shape[1])]
            y_interp = np.array([f(t_intervention) for f in interpolators])

            y0 = np.concatenate((
                y_interp,
                [pint * y_interp[0], pint * y_interp[1]]
            ))
            y0[0] = (1 - pint) * y_interp[0]
            y0[1] = (1 - pint) * y_interp[1]

        treated_values = treated_model.simulate(params[:8] + [effp, effc, pint, pint], treated_time, y0)
        cum_inf = treated_values[-1, 22] + treated_values[-1, 23]
        cum_infs.append(cum_inf)

    median_treated = np.median(cum_infs)
    percent_averted = 100 * (1 - median_treated / median_untreated_infections)
    return max(0, percent_averted)

# Execute simulations in parallel
param_grid = [(t, p) for t in t_interventions for p in pint_values]
results = Parallel(n_jobs=-1)(
    delayed(simulate_combination)(t, p) for t, p in tqdm(param_grid, desc='Simulating treated grid')
)

# Save result
heatmap_matrix = np.array(results).reshape(len(t_interventions), len(pint_values))
heatmap_df = pd.DataFrame(heatmap_matrix, index=np.round(t_interventions, 2), columns=np.round(pint_values, 2))
heatmap_df.index.name = 'intervention_start_day'
heatmap_df.columns.name = 'intervention_coverage'

output_file = '../data/starttime_coverage_percent_infections_averted.csv'
heatmap_df.to_csv(output_file)

Figure 3C

In [ ]:
all_untreated_passengers = []
all_untreated_crew = []

for _ in range(n_simulations):
    idx = np.random.randint(0, inferred_parameters.shape[0])
    selected_params = inferred_parameters.iloc[idx].to_list()
    untreated_values = untreated_model.simulate(selected_params[:8], times)
    cumul_infected_passengers = untreated_values[-1,22]
    cumul_infected_crew = untreated_values[-1,23]

    all_untreated_passengers.append(cumul_infected_passengers)
    all_untreated_crew.append(cumul_infected_crew)

all_untreated_passengers = np.array(all_untreated_passengers)
all_untreated_crew = np.array(all_untreated_crew)
median_passengers = np.median(all_untreated_passengers, axis=0)
median_crew = np.median(all_untreated_crew, axis=0)

coverage_array = np.linspace(0, 1, 50)
efficacy_array = np.linspace(0, 1, 50)

# Sample parameter sets in advance
sampled_indices = np.random.randint(0, inferred_parameters.shape[0], size=n_simulations)
sampled_parameters = [inferred_parameters.iloc[idx].to_list() for idx in sampled_indices]

def simulate_model(eff, pint, sampled_parameters, nu, times):
    all_treated_passengers = []
    all_treated_crew = []
    for params in sampled_parameters:
        treated_values = treated_model.simulate(params[:8] + [eff, eff, pint, pint], times)
        
        cumul_infected_treated_passengers = treated_values[-1,22]
        cumul_infected_treated_crew = treated_values[-1,23]

        all_treated_passengers.append(cumul_infected_treated_passengers)
        all_treated_crew.append(cumul_infected_treated_crew)
    
    median_treated_passengers = np.median(all_treated_passengers)
    median_treated_crew = np.median(all_treated_crew)
    return eff, pint, median_treated_passengers, median_treated_crew


# Prepare inputs for parallel processing
task_inputs = [(eff, pint, sampled_parameters, nu, times, n_simulations) 
               for eff in efficacy_array 
               for pint in coverage_array]

# Execute simulations in parallel
results = Parallel(n_jobs=-1)(delayed(simulate_model)(eff, pint, sampled_parameters, nu, times) for eff in efficacy_array for pint in coverage_array)

# Convert results to a DataFrame
results_df = pd.DataFrame(results, columns=['Efficacy', 'Coverage', 'Median Infected Passengers', 'Median Infected Crew'])

results_df.to_csv('../data/combined_coverage_efficacy_sensitivity_analysis.csv')

df = pd.read_csv('../data/combined_coverage_efficacy_sensitivity_analysis.csv')
filtered_df = df.loc[(df['Coverage'] == 0)]
total_untreated_infections = filtered_df.loc[filtered_df.index[0], 'Median Infected Passengers'] + filtered_df.loc[filtered_df.index[0], 'Median Infected Crew']
df['Percentage Averted'] = 1 - (df['Median Infected Passengers'] + df['Median Infected Crew'])/total_untreated_infections
df['Total infections averted'] = total_untreated_infections - (df['Median Infected Passengers'] + df['Median Infected Crew'])
output_file = '../data/efficacy_coverage_percent_infections_averted.csv'
df.to_csv(output_file, index=False)

Figure 3D

In [ ]:
# Pre-sample parameter sets to ensure both models use the same parameters for each simulation
sampled_indices = np.random.randint(0, inferred_parameters.shape[0], size=n_simulations)
sampled_parameters = [inferred_parameters.iloc[idx].to_list() for idx in sampled_indices]

coverage_array = [(1, 1), (1, 0), (0, 1), (0.5, 0.5)]
efficacy_array = np.linspace(0, 1, 100)

def simulate_models(params, times):
    untreated_values = untreated_model.simulate(params[:8], times)
    cumul_infected_untreated_passengers = untreated_values[-1,22]
    cumul_infected_untreated_crew = untreated_values[-1,23]
    total_untreated_infections = cumul_infected_untreated_passengers + cumul_infected_untreated_crew

    averted_results = {}
    for eff in efficacy_array:
        for coverage in coverage_array:
            treated_values = treated_model.simulate(params[:8] + [eff, eff, coverage[0], coverage[1]], times)
            cumul_infected_treated_passengers = treated_values[-1,22]
            cumul_infected_treated_crew = treated_values[-1,23]
            total_treated_infections = cumul_infected_treated_passengers + cumul_infected_treated_crew

            infections_averted_percentage = 100 * (total_untreated_infections - total_treated_infections) / total_untreated_infections if total_untreated_infections > 0 else 0
            
            averted_results.setdefault((eff, coverage[0], coverage[1]), []).append(infections_averted_percentage)

    return averted_results

# Execute simulations in parallel
all_results = Parallel(n_jobs=-1)(delayed(simulate_models)(params, times) for params in sampled_parameters)

# Combine results from all simulations
combined_results = {}
for result in all_results:
    for key, values in result.items():
        combined_results.setdefault(key, []).extend(values)

# Calculate percentiles for each combination of efficacy and coverage
percentile_results = []
for key, values in combined_results.items():
    efficacy, passenger_coverage, crew_coverage = key
    median_averted = np.percentile(values, 50)
    p5_averted = np.percentile(values, 2.5)
    p95_averted = np.percentile(values, 97.5)
    percentile_results.append((efficacy, passenger_coverage, crew_coverage, median_averted, p5_averted, p95_averted))

# Convert results to a DataFrame
columns = ['Efficacy', 'Passenger Coverage', 'Crew Coverage', 'Median Percentage Infections Averted', '2.5th Percentile Percentage Infections Averted', '97.5th Percentile Percentage Infections Averted']
results_df = pd.DataFrame(percentile_results, columns=columns)
output_file = 'passengers_crew_infections_averted_efficacy.csv'
results_df.to_csv(output_file, index=False)